###  **Question 3: IAQF Academic Competition**


*Imports*

In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf
from datetime import date
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

##### **a) What are the implications of the noted concentration for that strategy?**

**Data**

*Reading*

In [2]:
sector_etfs = {
    'Baseline': {
        "Large-Cap": "SPY",
        "Small-Cap": "^SP600"
    },
    "Technology": {
        "Large-Cap": "XLK",
        "Small-Cap": "PSCT"
    },
    "Healthcare": {
        "Large-Cap": "XLV",
        "Small-Cap": "PSCH"
    },
    "Financials": {
        "Large-Cap": "XLF",
        "Small-Cap": "PSCF"
    },
    "Consumer Discretionary": {
        "Large-Cap": "XLY",
        "Small-Cap": "PSCD"
    },
    "Consumer Staples": {
        "Large-Cap": "XLP",
        "Small-Cap": "PSCC"
    },
    "Industrials": {
        "Large-Cap": "XLI",
        "Small-Cap": "PSCI"
    },
    "Energy": {
        "Large-Cap": "XLE",
        "Small-Cap": "PSCE"
    },
    "Materials": {
        "Large-Cap": "XLB",
        "Small-Cap": "PSCM"
    },
    "Utilities": {
        "Large-Cap": "XLU",
        "Small-Cap": "PSCU"
    },
    "Real Estate": {
        "Large-Cap": "XLRE",
        "Small-Cap": "ROOF"
    }
}

In [3]:
tickers = [_ for x in sector_etfs.values() for _ in x.values()]
start_date = date(2010,1,1)
end_date = date(2025,1,1)

# ETF Returns
prices = yf.download(tickers = tickers,start = start_date,end = end_date)['Close']

ff_data = pd.read_csv('../extra-data/FF_3Factor.CSV',skiprows = 4,index_col=0)
ff5_data = pd.read_csv('../extra-data/FF_5Factor.csv',index_col=0)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  22 of 22 completed


*Cleaning*

In [4]:
returns = prices.pct_change().dropna()

# Clean FF Data
ff_data = ff_data.iloc[:-1]
ff_data.index = pd.to_datetime(ff_data.index).date

ff5_data.index = pd.to_datetime(ff5_data.index).date

/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_7590/1153697555.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


##### *i)* Baseline Strategy


Baseline strategy is underperforming, show how this is connected to tech exposure

In [5]:
baseline_strategy = pd.DataFrame(returns['^SP600'] - returns['SPY'],columns=['daily_returns'])
baseline_strategy['cumulative_returns'] = (1 + baseline_strategy['daily_returns']).cumprod()

##### *ii)* Industry Sector Basis

Show the strategy performance at a sector basis

In [6]:
sector_baseline_returns = {}

for sector,(large,small) in sector_etfs.items():
    sector_baseline_returns[sector] = (1 + returns[sector_etfs[sector][small]] - returns[sector_etfs[sector][large]]).cumprod()

sector_baseline_returns = pd.DataFrame(sector_baseline_returns)

##### *iii)* Risk Attribution (Fama-French)

In [7]:
ff5_data.columns

Index(['mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd'], dtype='object')

In [8]:
import statsmodels.api as sm
# Merge datasets
df = baseline_strategy.join(ff5_data, how='inner')

# Calculate excess returns
df['Excess_Return'] = df['daily_returns'] - df['rf']

# Run regression
X = df[['smb', 'hml', 'rmw', 'cma','umd']]
X = sm.add_constant(X)  # Add intercept
y = df['Excess_Return']

model = sm.OLS(y, X).fit()
print(model.summary())

# Extract risk contribution
betas = model.params[1:]  # Exclude intercept
factor_vols = df[['smb', 'hml', 'rmw', 'cma','umd']].std()  # Factor standard deviations

# Compute risk contribution
risk_contributions = (betas**2 * factor_vols**2) / sum(betas**2 * factor_vols**2)
print("Risk Attribution:\n", risk_contributions)

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     2087.
Date:                Fri, 21 Feb 2025   Prob (F-statistic):               0.00
Time:                        13:28:02   Log-Likelihood:                 3557.5
No. Observations:                 753   AIC:                            -7103.
Df Residuals:                     747   BIC:                            -7075.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003   7.88e-05     -3.639      0.0

#### **Plots**

*i)* Baseline Strategy Performance

In [9]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = baseline_strategy.index,
        y = baseline_strategy['cumulative_returns'],
        name = 'Baseline Strategy'
    )
)

fig.update_layout(title = 'Baseline Long Small Cap, Short Large Cap Strategy')
fig.update_yaxes(title = 'Cumulative Returns')
fig.update_xaxes(title = 'Date')
fig.show()

*ii)* Strategy at Sector Level

In [10]:
fig = go.Figure()


for strategy in sector_baseline_returns:

    fig.add_trace(
        go.Scatter(
            x = sector_baseline_returns.index,
            y = sector_baseline_returns[strategy],
            name = strategy
        )
    )

fig.update_layout(title = 'Strategy Sector Level Returns',
    legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),)
fig.update_yaxes(title = 'Cumulative Returns')
fig.update_xaxes(title = 'Date')
fig.show()

##### **b) How would you modify the trading strategy to correct for any implication you may have found?**

In [11]:
small_cap = ['PSCT', 'PSCH', 'PSCF','PSCD', 'PSCC', 'PSCI', 'PSCE', 'PSCM','PSCU','ROOF']
large_cap = ['XLK', 'XLV', 'XLF', 'XLY', 'XLP', 'XLI', 'XLE', 'XLB', 'XLU', 'XLRE']

start_date = date(2020,1,1)
end_date = date(2025,1,1)

data = yf.download(tickers = small_cap + large_cap,start = start_date, end = end_date)['Close']

returns = data.pct_change().dropna()

[*********************100%***********************]  20 of 20 completed
/var/folders/__/wgnmk9j14kl1krf80l2c6bfw0000gn/T/ipykernel_7590/3941965587.py:9: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [12]:
class Momentum:

    def __init__(self, data, returns, lookback, rebalance, type = 'Equal'):
        # Inputs
        self.data = data
        self.returns = returns
        self.lookback = lookback
        self.rebalance = rebalance
        self.type = type

        # Stats
        self.weights = pd.Series(0.0, index=self.returns.columns, dtype=float)
        self.portfolio_returns = pd.Series(0.0, index=self.returns.index, dtype=float)

    def run(self):

        for t in range(self.lookback, len(self.returns.index)):
            # Compute portfolio return for the day
            daily_return = (self.weights * self.returns.iloc[t]).sum()
            self.portfolio_returns.iloc[t] = daily_return

            if t % self.rebalance == 0:

                # Get window returns
                small_cap_window_returns = self.returns.iloc[t - self.lookback:t,][small_cap]
                large_cap_window_returns = self.returns.iloc[t - self.lookback:t,][large_cap]

                # Calculate momentum, long and short threshold
                momentum_small_cap = self.signal(small_cap_window_returns)
                momentum_large_cap = self.signal(large_cap_window_returns)

                # Select assets in the top and bottom deciles
                long_assets = momentum_small_cap.nlargest(3).index
                short_assets = momentum_large_cap.nsmallest(3).index

                # Equally Weighted Portfolio
                self.weights = pd.Series(0.0, index=self.returns.columns, dtype=float)
                self.weights[long_assets] = 1 / len(long_assets) if len(long_assets) > 0 else 0
                self.weights[short_assets] = -1 / len(short_assets) if len(short_assets) > 0 else 0

            

        # Clean + Compute Stats
        self._clean_returns()
        self._compute_stats()

    def signal(self, window_returns):
        """Calculate Momentum Signals"""
        if self.type == 'Equal':

            # Calculate momentum (cumulative returns over the lookback period)
            momentum = (1 + window_returns).prod() - 1 / np.sqrt(window_returns.var())
    
            return momentum
        
    def _clean_returns(self):
        """Clean Portfolio Returns"""

        _df = pd.DataFrame(self.portfolio_returns,columns=['daily_return'])
        _df['cumulative_return'] = (1 + _df['daily_return']).cumprod()
        
        self.portfolio_returns = _df

    def _compute_stats(self):
        """Compute Portfolio Stats"""
        
        # Annualized Return
        annual_ret = self.portfolio_returns['daily_return'].mean()*252

        # Volatility
        annual_vol = self.portfolio_returns['daily_return'].std()*np.sqrt(252)

        # Sharpe Ratio
        sharpe_ratio = annual_ret / annual_vol

        self.stats = {'Sharpe Ratio': round(sharpe_ratio,2),
                      'Annual Return': round(annual_ret,2),
                        'Annual Volatility': round(annual_vol,2)}

In [13]:
m = Momentum(data,returns,120,30,'Equal')
m.run()
portfolio_returns = m.portfolio_returns

In [14]:
import statsmodels.api as sm
# Merge datasets
df = portfolio_returns.join(ff5_data, how='inner')

# Calculate excess returns
df['Excess_Return'] = df['daily_return'] - df['rf']

# Run regression
X = df[['smb', 'hml', 'rmw', 'cma','umd']]
X = sm.add_constant(X)  # Add intercept
y = df['Excess_Return']

model = sm.OLS(y, X).fit()
print(model.summary())

# Extract risk contribution
betas = model.params[1:]  # Exclude intercept
factor_vols = df[['smb', 'hml', 'rmw', 'cma','umd']].std()  # Factor standard deviations

# Compute risk contribution
risk_contributions = (betas**2 * factor_vols**2) / sum(betas**2 * factor_vols**2)
print("Risk Attribution:\n", risk_contributions)

                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     174.0
Date:                Fri, 21 Feb 2025   Prob (F-statistic):          1.23e-122
Time:                        13:28:08   Log-Likelihood:                 2345.2
No. Observations:                 753   AIC:                            -4678.
Df Residuals:                     747   BIC:                            -4651.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.000      1.204      0.2

##### **c) Can you provide a backtest comparison between the standard strategy and the modified strategy?**

In [21]:
from sklearn.decomposition import PCA

lreturns = np.log(data).diff().dropna()
lreturns = (lreturns - lreturns.mean()) / lreturns.std()

pca = PCA(n_components=5)  # Using top 5 components
pca.fit(lreturns)

# Get the loadings (weights) of the principal components
loadings = pca.components_

# Create a DataFrame to show the loadings with the stock tickers as the index
loadings_df = pd.DataFrame(loadings.T, columns=[f'PC{i+1}' for i in range(loadings.shape[0])], index=lreturns.columns)

In [27]:
fig = go.Figure()


for pc in loadings_df.T.columns:
    fig.add_trace(
        go.Bar(
            x = loadings_df.T.index,
            y = loadings_df.T[pc]**2,
            name = pc
        )
    )


fig.update_layout(title = 'PC Loadings, Log Returns')
fig.update_yaxes(title = 'Loading^2')
fig.show()